In [3]:
from bs4 import BeautifulSoup as bs
from bs4 import NavigableString, Tag
from glob import glob
import spacy

py_nlp = spacy.load ("en_core_web_lg")

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [4]:
def html2bio(html_path):
    with open(html_path) as f:
        
        html = f.read()
        
        if '***output***' in html.lower():
            html = html[html.lower().index('***output***')+len('***output***')+1:]
        if 'output:' in html.lower():
            html = html[html.lower().index('output:')+len('output:')+1:]
        if 'output text' in html.lower():
            html = html[html.lower().index('output text')+len('output text')+1:]
        if '***Highlighted Text***'  in html.lower():
            html = html[html.lower().index('***Highlighted Text***')+len('***Highlighted Text***')+1:]
        if '<body>' in html:
            html = html[html.index('<body>')+6:html.index('</body>')]
        if '<p>' in html:
            html = html[html.index('<p>')+3:html.index('</p>')]
            
        #print (html_path)
        #print (html,'\n')
        
        # Parse HTML using BeautifulSoup
        soup = bs(html, "html.parser")

        # Extract text under 'p' tags and convert to BIO format
        bio_format = []
        

        for child in soup.children:
            if isinstance(child, NavigableString):
                for word in child.split():
                    bio_format.append(f"{word}\tO\n")
            elif isinstance(child, Tag):
                words = py_nlp (child.get_text())
                try:
                    entity = child.attrs['class'][0]
                except:
                    entity = 'O'
                if len(words) != 0:
                    if entity != 'O' and entity in ['problem','treatment','test']:
                        bio_format.append(f"{words[0]}\tB-{entity}\n")
                        for word in words[1:]:
                            bio_format.append(f"{word}\tI-{entity}\n")
                    else:
                        bio_format.append(f"{words[0]}\tO\n")
                        for word in words[1:]:
                            bio_format.append(f"{word}\tO\n")
    return bio_format

In [5]:
def get_performance():
    all_tags = []
    all_tokens = []
    gold_tags = []

    for file in files:
        file_name = file.split('/')[-1].split('.')[0]
        with open(file) as f_gold:
            lines = f_gold.readlines()
            tokens = [line.strip().split('\t')[0] for line in lines]
            tags = [line.strip().split('\t')[-1] for line in lines]

            prediction = f'./data/evaluation_results/{file_name}.html'

            bio_2 = html2bio(prediction)
            all_tokens += tokens

            for i,token in enumerate(tokens):
                if token != '':
                    match = False
                    for i2 in range(i,-1,-1):
                        try:
                            token_2,tag_2 = bio_2[i2].strip().split('\t')
                        except:
                            token_2,tag_2 = None, None
                        
                        if token_2!=None:
                            if token in token_2 or token_2 in token:
                                match = True
                                break

                    if not match:
                        tag_2 = 'O'
                else:
                    tag_2 = ''
                #print (token,tags[i],tag_2)
                gold_tags.append(tags[i])
                all_tags.append(tag_2)
    with open('./output/merged_gold_pre.bio','w') as fg:
        for i,(token,gold_tag,all_tag) in enumerate(zip(all_tokens,gold_tags,all_tags)):
            if token!='':

                fg.write(f'{token}\t{gold_tag}\t{all_tag}\n')
            else:
                fg.write(f'\n')
    !python ./evaluate.py -lf ./output/merged_gold_pre.bio

In [ ]:
GPT = 3.5
prompt = 'baseline'
get_performance(GPT,prompt)
prompt = 'baseline_definition_guideline'
get_performance(GPT,prompt)
prompt = 'baseline_definition_guideline_erroranalysis'
get_performance(GPT,prompt)
prompt = 'baseline_definition_guideline_erroranalysis_1shot'
get_performance(GPT,prompt)
prompt = 'baseline_definition_guideline_erroranalysis_5shot'
get_performance(GPT,prompt)